<center>
<img src="https://tensorflowkorea.files.wordpress.com/2020/12/4.-e18492e185a9e186abe1848ce185a1-e18480e185a9e186bce18487e185aee18492e185a1e18482e185b3e186ab-e18486e185a5e18489e185b5e186abe18485e185a5e18482e185b5e186bce18483e185b5e186b8e18485e185a5e.png?w=972" width="200" height="200"><br>
</center>

# chapter4.  다양한 분류 알고리즘

## 04-1 로지스틱 회귀

### - 럭키백의 확률

이번 예제는 7가지 생선 중 럭키백에 들어가 있는 생선의 확률을 구하는 것이다. 럭키백에 들어간 생선의 크기, 무게 등이 주어졌을 때 7개 생선에 대한 확률을 출력해 볼 것이다.

### - 데이터 준비하기

판다스의 read_csv() 함수로 csv 파일을 데이터프레임으로 변환한 다음 head() 메서드로 처음 5개의 행을 출력해보자.
데이터 출처 : https://github.com/rickiepark/hg-mldl

In [1]:
import pandas as pd
fish = pd.read_csv('/home/jaeyoon89/hg-mldl/fish.csv')

In [2]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


어떤 종류의 생선이 있는지 Species열에서 고유한 값을 출력해 보자. 판다스의 unique() 함수를 사용하면 간단하다.

In [4]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


이 데이터프레임에서 Species 열을 타깃으로 만들고 나머지 5개 열은 입력 데이터로 사용하자. 데이터프레임에서 열을 선택하는 방법은 간단하다. 데이터프레임에서 원하는 열을 리스트로 나열하면 된다. Species 열을 빼고 나머지 5개 열을 선택해 보자.

In [5]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

데이터프레임에서 여러 열을 선택하면 새로운 데이터프레임이 반환된다. 이를 to_numpy() 메서드로 넘파이 배열로 바꾸어 fish_input에 저장했다. fish_input에 처음 5개의 행을 출력해보자.

In [6]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


이제 동일한 방식으로 타깃 데이터를 만들자.

In [7]:
fish_target = fish['Species'].to_numpy()

앞서 배웠듯이 머신러닝에서는 기본으로 데이터 세트가 2개 필요하다.

In [8]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

그 다음 사이킷런의 StandardScaler 클래스를 사용해 훈련 세트와 테스트 세트를 표준화 전처리하자. 여기에서도 훈련 세트의 통계 값으로 테스트 세트를 변환해야 한다.

In [10]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### - k-최근접 이웃 분류기의 확률 예측

앞서 배웠듯이 사이킷런의 KNeighborsClassfier 클래스 객체를 만들고 훈련 세트로 모델을 훈련한 다음 훈련 세트와 테스트 세트의 점수를 확인해 보자. k를 3으로 지정하여 사용해보자.

In [11]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


여기에선 클래스 확률을 배우는 것이 목적이라 위 점수에 대해서는 잊도록 하자.

앞서 fish 데이터프레임에서 7개의 생선이 있었다. 타깃 데이터가 Species이기 때문에 훈련 세트와 테스트 세트의 타깃 데이터에도 7개의 생선 종류가 들어가 있다. 이렇게 타깃 데이터에 2개 이상의 클래스가 포함된 문제를 다중 분류라고 한다. 앞서 이진 분류를 배웠듯이 다중 분류에서도 타깃값을 숫자로 바꾸어 입력할 수 있지만 사이킷런에서는 편리하게 문자열로 된 타깃값을 그대로 사용할 수 있다.

하지만 주의할 점은 타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순서가 된다. 따라서 pd.unique로 출력했던 순서와 다르다. KNeighborsClassifier에서 정렬된 타깃값은 classes_속성에 저장되어 있다.

In [12]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


이번엔 predict() 메서드를 이용해 타깃값으로 예측을 출력해보자. 테스트 세트에 처음 5개 샘플의 타깃값을 예측해보자.

In [13]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


위 5개의 대한 예측은 어떤 확률로 만들어 졌을까? 사이킷런의 분류 모델은 predict_proba() 메서드로 클래스별 확률값을 반환한다. 테스트 세트에 있는 처음 5개의 샘플에 대한 확률을 출력해 보자. 넘파이 round() 함수는 기본으로 소수점 첫째 자리에서 반올림 하는데, decimals 매개변수로 유지할 소수점 아래 자릿수를 지정할 수 있다.

In [14]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


predict_proba() 메서드의 출력 순서는 앞서 보았던 classes_ 속성과 같다. 